In [2]:
# EDA
import pandas as pd
import time

In [3]:
# Load the hotel reviews from CSV
print("Loading data file now, this could take a while depending on file size")
start = time.time()
df = pd.read_csv('../data/Hotel_Reviews.csv')
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds")


Loading data file now, this could take a while depending on file size
Loading took 2.98 seconds


In [4]:
# What shape is the data (rows, columns)?
print("The shape of the data (rows, cols) is " + str(df.shape))


The shape of the data (rows, cols) is (515738, 17)


In [12]:
df[:2]

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968


In [4]:
# value_counts() creates a Series object that has index and values
#                in this case, the country and the frequency they occur in reviewer nationality
nationality_freq = df["Reviewer_Nationality"].value_counts()
nationality_freq

Reviewer_Nationality
 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
                               ...  
 Cape Verde                        1
 Northern Mariana Islands          1
 Tuvalu                            1
 Guinea                            1
 Palau                             1
Name: count, Length: 227, dtype: int64

In [9]:
# What reviewer nationality is the most common in the dataset?
print("The highest frequency reviewer nationality is " + str(nationality_freq.index[0]).strip() + " with " + str(nationality_freq[0]) + " reviews.")


The highest frequency reviewer nationality is United Kingdom with 245246 reviews.


In [10]:
# What is the top 10 most common nationalities and their frequencies?
print("The top 10 highest frequency reviewer nationalities are:")
print(nationality_freq[0:10].to_string())


The top 10 highest frequency reviewer nationalities are:
Reviewer_Nationality
 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
 Saudi Arabia                   8951
 Netherlands                    8772
 Switzerland                    8678
 Germany                        7941
 Canada                         7894


In [11]:
# How many unique nationalities are there?
print("There are " + str(nationality_freq.index.size) + " unique nationalities in the dataset")


There are 227 unique nationalities in the dataset


In [13]:
# What was the most frequently reviewed hotel for the top 10 nationalities - print the hotel and number of reviews
for nat in nationality_freq[:10].index:
   # First, extract all the rows that match the criteria into a new dataframe
   nat_df = df[df["Reviewer_Nationality"] == nat]   
   # Now get the hotel freq
   freq = nat_df["Hotel_Name"].value_counts()
   print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq[0]) + " reviews.") 


The most reviewed hotel for United Kingdom was Britannia International Hotel Canary Wharf with 3833 reviews.
The most reviewed hotel for United States of America was Hotel Esther a with 423 reviews.
The most reviewed hotel for Australia was Park Plaza Westminster Bridge London with 167 reviews.
The most reviewed hotel for Ireland was Copthorne Tara Hotel London Kensington with 239 reviews.
The most reviewed hotel for United Arab Emirates was Millennium Hotel London Knightsbridge with 129 reviews.
The most reviewed hotel for Saudi Arabia was The Cumberland A Guoman Hotel with 142 reviews.
The most reviewed hotel for Netherlands was Jaz Amsterdam with 97 reviews.
The most reviewed hotel for Switzerland was Hotel Da Vinci with 97 reviews.
The most reviewed hotel for Germany was Hotel Da Vinci with 86 reviews.
The most reviewed hotel for Canada was St James Court A Taj Hotel London with 61 reviews.


In [5]:
# How many reviews are there per hotel (frequency count of hotel) and do the results match the value in `Total_Number_of_Reviews`?
# First create a new dataframe based on the old one, removing the uneeded columns
hotel_freq_df = df.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", "Reviewer_Nationality", "Negative_Review", "Review_Total_Negative_Word_Counts", "Positive_Review", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "Tags", "days_since_review", "lat", "lng"], axis = 1)

In [6]:
hotel_freq_df[:5]

,Hotel_Name,Total_Number_of_Reviews
0,Hotel Arena,1403
1,Hotel Arena,1403
2,Hotel Arena,1403
3,Hotel Arena,1403
4,Hotel Arena,1403


In [7]:
# Group the rows by Hotel_Name, count them and put the result in a new column Total_Reviews_Found
hotel_freq_df['Total_Reviews_Found'] = hotel_freq_df.groupby('Hotel_Name').transform('count')

In [8]:

# Get rid of all the duplicated rows
hotel_freq_df = hotel_freq_df.drop_duplicates(subset = ["Hotel_Name"])
print()
print(hotel_freq_df.to_string())
print(str(hotel_freq_df.shape))


                                                          Hotel_Name  Total_Number_of_Reviews  Total_Reviews_Found
0                                                        Hotel Arena                     1403                  405
405                                                 K K Hotel George                     1831                  566
971                                          Apex Temple Court Hotel                     2619                 1037
2008                                The Park Grand London Paddington                     4380                 1770
3778                                             Monhotel Lounge SPA                      171                   35
3813                                              Kube Hotel Ice Bar                      197                   19
3832                                            The Principal London                     3150                 1425
5257                                   Park Plaza County Hall London           

In [9]:
def get_difference_review_avg(row):
    return row["Average_Score"] - row["Calc_Average_Score"]

In [10]:
# While there is an `Average_Score` for each hotel according to the dataset, 
# you can also calculate an average score (getting the average of all reviewer scores in the dataset for each hotel)
# Add a new column to your dataframe with the column header `Calc_Average_Score` that contains that calculated average. 
df['Calc_Average_Score'] = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)
# Add a new column with the difference between the two average scores
df["Average_Score_Difference"] = df.apply(get_difference_review_avg, axis = 1)
# Create a df without all the duplicates of Hotel_Name (so only 1 row per hotel)
review_scores_df = df.drop_duplicates(subset = ["Hotel_Name"])
# Sort the dataframe to find the lowest and highest average score difference
review_scores_df = review_scores_df.sort_values(by=["Average_Score_Difference"])
print(review_scores_df[["Average_Score_Difference", "Average_Score", "Calc_Average_Score", "Hotel_Name"]])
# Do any hotels have the same (rounded to 1 decimal place) `Average_Score` and `Calc_Average_Score`?


        Average_Score_Difference  Average_Score  Calc_Average_Score   
495945                      -0.8            7.7                 8.5  \
111027                      -0.7            8.8                 9.5   
43688                       -0.7            7.5                 8.2   
178253                      -0.7            7.9                 8.6   
218258                      -0.5            7.0                 7.5   
...                          ...            ...                 ...   
151416                       0.7            7.8                 7.1   
22189                        0.8            7.1                 6.3   
250308                       0.9            8.6                 7.7   
68936                        0.9            6.8                 5.9   
3813                         1.3            7.2                 5.9   

                                               Hotel_Name  
495945                         Best Western Hotel Astoria  
111027  Hotel Stendhal Plac

In [11]:
display(review_scores_df[["Average_Score_Difference", "Average_Score", "Calc_Average_Score", "Hotel_Name"]])

,Average_Score_Difference,Average_Score,Calc_Average_Score,Hotel_Name
495945,-0.8,7.7,8.5,Best Western Hotel Astoria
111027,-0.7,8.8,9.5,Hotel Stendhal Place Vend me Paris MGallery by...
43688,-0.7,7.5,8.2,Mercure Paris Porte d Orleans
178253,-0.7,7.9,8.6,Renaissance Paris Vendome Hotel
218258,-0.5,7.0,7.5,Hotel Royal Elys es
...,...,...,...,...
151416,0.7,7.8,7.1,Best Western Allegro Nation
22189,0.8,7.1,6.3,Holiday Inn Paris Montparnasse Pasteur
250308,0.9,8.6,7.7,MARQUIS Faubourg St Honor Relais Ch teaux
68936,0.9,6.8,5.9,Villa Eugenie


In [25]:
# with lambdas:
start = time.time()
no_negative_reviews = df.apply(lambda x: True if x['Negative_Review'] == "No Negative" else False , axis=1)
print("Number of No Negative reviews: " + str(len(no_negative_reviews[no_negative_reviews == True].index)))

no_positive_reviews = df.apply(lambda x: True if x['Positive_Review'] == "No Positive" else False , axis=1)
print("Number of No Positive reviews: " + str(len(no_positive_reviews[no_positive_reviews == True].index)))

both_no_reviews = df.apply(lambda x: True if x['Negative_Review'] == "No Negative" and x['Positive_Review'] == "No Positive" else False , axis=1)
print("Number of both No Negative and No Positive reviews: " + str(len(both_no_reviews[both_no_reviews == True].index)))
end = time.time()
print("Lambdas took " + str(round(end - start, 2)) + " seconds")


Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Lambdas took 7.43 seconds


In [24]:
# 也可以直接用如下方法(更快)
start = time.time()
print(len(df[df['Negative_Review'] == "No Negative"].index))
print(len(df[df['Positive_Review'] == "No Positive"].index))
print(len(df[df['Negative_Review'] == "No Negative"][ df['Positive_Review'] == "No Positive"].index))
end = time.time()
print("Lambdas took " + str(round(end - start, 2)) + " seconds")

127890
35946
127
Lambdas took 0.23 seconds


/var/folders/c5/6z8x0w0s2mv90f5wxjdl0x440000gn/T/ipykernel_63796/1739567158.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(df[df['Negative_Review'] == "No Negative"][ df['Positive_Review'] == "No Positive"].index))


In [26]:
# 再一种方法
# without lambdas (using a mixture of notations to show you can use both)
start = time.time()
no_negative_reviews = sum(df.Negative_Review == "No Negative")
print("Number of No Negative reviews: " + str(no_negative_reviews))

no_positive_reviews = sum(df["Positive_Review"] == "No Positive")
print("Number of No Positive reviews: " + str(no_positive_reviews))

both_no_reviews = sum((df.Negative_Review == "No Negative") & (df.Positive_Review == "No Positive"))
print("Number of both No Negative and No Positive reviews: " + str(both_no_reviews))

end = time.time()
print("Sum took " + str(round(end - start, 2)) + " seconds")

Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Sum took 0.27 seconds
